In [1]:
import pandas as pd
import numpy as np
import re
from random import random
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import torch
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
from simpletransformers.classification import MultiLabelClassificationModel, ClassificationModel
from sklearn.preprocessing import MultiLabelBinarizer

I0615 13:43:31.277418 140567140800320 file_utils.py:39] PyTorch version 1.5.0 available.


In [3]:
from simpletransformers.language_modeling import LanguageModelingModel

## Read input data

In [4]:
# df = pd.read_json('./source/train_gold.json', lines=True)
df = pd.read_json('./preprocessed/preprocess_train.json', lines=True)
df_new = df.copy()
df_new.head()

,idx,text,reply,categories
0,0,we can all agree that any song by Niall Horan .,oui oui,[yes]
1,1,Will you be install # Scotty from marketing ...,,[no]
2,2,Growing up my mum would call me a Nigga despit...,And he joins in ? ? ? Pour some hot grits o...,[smh]
3,3,Rest your head on my chest when the world feel...,face with tear of joy,[wink]
4,4,Imagine Will Hernandez and Wills both doing a ...,,[yes]


In [5]:
categories_type = pd.read_json('./source/categories.json', lines=True)
categories_mapping = {v[0]: k for k, v in categories_type.to_dict('list').items()}

In [6]:
df_dev_ori = pd.read_json('./source/dev_unlabeled.json', lines=True)
df_dev = pd.read_json('./preprocessed/preprocess_dev.json', lines=True)
df_dev_result = df_dev.copy()[['text', 'reply']]
df_dev_result.head()

,text,reply
0,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans
1,After interviewing with a few incredible peopl...,CONGRATS ! ! ! ! !
2,I know GTC festival not happening next month b...,
3,"Lordy , my daughter just said , "" I wonde...",
4,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...


In [7]:
df_test_ori = pd.read_json('./source/test_unlabeled.json', lines=True)
df_test = pd.read_json('./preprocessed/preprocess_test.json', lines=True)
df_test_result = df_test.copy()[['text', 'reply']]
df_test_result.head()

,text,reply
0,@ Youngdeji - I think if uzi and carti dro...,
1,For the third year in a row we are discussing ...,
2,dababy album sounds like it was made for nigga...,That ' s why you buy it .
3,Majority of Indians do not watch any sport oth...,@ ZairaWasimmm got a great story because of t...
4,everybody is just now listen to @ madisonbeer...,


### Use multi-hot encoding and change column name

In [8]:
mlb = MultiLabelBinarizer()
multi_hot = mlb.fit_transform(df_new['categories'].values)
multi_hot_list = [list(_) for _ in multi_hot]

In [9]:
df_new['labels'] = multi_hot_list
df_new = df_new[['text', 'reply', 'labels']]
df_new.columns = ['text_a', 'text_b', 'labels']
df_dev_result.columns = ['text_a', 'text_b']
df_test_result.columns = ['text_a', 'text_b']

## Training language model first, can imporve accuracy!

In [13]:
# Output data to train_file for training LM
df_forLM = df_new.copy()
# df_forLM['text'] = ['<s> '] + df_new['text_a'] + [' </s></s> '] + df_new['text_b'] + [' <s>']
df_forLM['text'] = ['[CLS] '] + df_new['text_a'] + [' [SEP] '] + df_new['text_b'] + [' [SEP]']
# df_forLM_dev = df_dev_result.copy()
# df_forLM_dev['text'] = ['<s> '] + df_dev_result['text_a'] + [' </s></s> '] + df_dev_result['text_b'] + [' <s>']
# df_forLM_test = df_test_result.copy()
# df_forLM_test['text'] = ['<s> '] + df_test_result['text_a'] + [' [SEP] '] + df_test_result['text_b']
# pd.concat([df_forLM['text'], df_forLM_dev['text'], df_forLM_test['text']])[1]

In [14]:
df_forLM['text'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')
# pd.concat([df_forLM['text'], df_forLM_dev['text']]).to_csv(r'LM/training_dev.txt', header=None, index=None, sep=' ')
# pd.concat([df_forLM['text'], df_forLM_dev['text'], df_forLM_test['text']]).to_csv(r'LM/training_dev_test.txt', header=None, index=None, sep=' ')

In [16]:
for i in range(1, 6):
    seq = 113
    block_size = 96
    train_lm_args = {
        "output_dir": "round2_blend_model/round2_LM/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "LM/outputs/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "block_size": block_size,
        "max_seq_length": seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,
    }
    MLR_seq = 113
#     model_lm = LanguageModelingModel("bert", "bert-base-uncased", args=train_lm_args)
    model_lm = LanguageModelingModel("bert", "bert-base-cased", args=train_lm_args)
#     model_lm = LanguageModelingModel("roberta", "roberta-base", args=train_lm_args)
    model_lm.train_model('LM/training.txt')
    train_args = {
        "output_dir": "round2_blend_model/round2_MLR/outputs_bert_cased_LM_training{}/".format(i),
        "cache_dir": "cache/",
        "best_model_dir": "model_results_gpu/outputs/best_model/",
        
        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": MLR_seq,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 4,
        "weight_decay": 0,
        "learning_rate": 4e-5,
        "adam_epsilon": 1e-8,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "save_model_every_epoch": False,

        "overwrite_output_dir": False,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": False,

        "use_early_stopping": True,
        "early_stopping_patience": 3,
        "early_stopping_delta": 0,
        "early_stopping_metric": "eval_loss",
        "early_stopping_metric_minimize": True,

        "manual_seed": None,
        "encoding": None,
    }
    model = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_LM/outputs_bert_cased_LM_training{}/'.format(i), num_labels=43, args=train_args)
#     model = MultiLabelClassificationModel('roberta', 'round2_blend_model/round2_LM/outputs_roberta_LM_training{}/'.format(i), num_labels=43, args=train_args)
    model.train_model(df_new)
    result, model_outputs, wrong_predictions = model.eval_model(df_new)

I0615 07:11:33.574813 140378694625088 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at cache/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I0615 07:11:34.557356 140378694625088 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at cache/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0615 07:11:34.557963 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 5

I0615 07:11:42.475920 140378694625088 language_modeling_utils.py:229]  Saving features into cached file cache/bert_cached_lm_94_training.txt


I0615 07:11:42.505619 140378694625088 language_modeling_model.py:473]  Training started


Running loss: 2.077152


Running loss: 2.671032


...

Running loss: 0.119589

I0615 09:56:57.806064 140378694625088 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/checkpoint-6000/config.json
I0615 09:56:58.016796 140378694625088 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/checkpoint-6000/pytorch_model.bin


Running loss: 0.094158

I0615 10:02:22.059819 140378694625088 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/checkpoint-8000/config.json
I0615 10:02:22.263452 140378694625088 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/checkpoint-8000/pytorch_model.bin
I0615 10:02:22.689396 140378694625088 configuration_utils.py:144] Configuration saved in round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/config.json


I0615 10:02:22.895425 140378694625088 modeling_utils.py:483] Model weights saved in round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/pytorch_model.bin
I0615 10:02:22.908523 140378694625088 classification_model.py:279]  Training of bert model complete. Saved to round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/.


## Predict dev results [pair sentance](https://simpletransformers.ai/docs/classification-data-formats/)

In [57]:
# text + [SEP] + reply
to_predict = []
for i in range(len(df_dev_result)):
    # for roberta format
    text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
#     for bert format
#     text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [58]:
predictions, raw_outputs = model.predict(to_predict)

I0613 16:38:13.649572 140669230749504 classification_model.py:801]  Converting to features started. Cache is not used.


In [59]:
total_predict = []
for eid, row in enumerate(raw_outputs):
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
    total_predict.append(predict_class)

In [60]:
df_dev_ori['categories'] = total_predict

In [61]:
df_dev_ori.head()

,idx,text,reply,categories
0,32000,"Drop your cash app, use hashtag #BailoutHumansNow",$tyratomaro #BailoutHumans,"[applause, dance, good_luck, please, thank_you..."
1,32001,After interviewing with a few incredible peopl...,CONGRATS!!!!!,"[agree, applause, slow_clap, thumbs_up, win, yes]"
2,32002,I know GTC festival not happening next month b...,,"[facepalm, no, seriously, shrug, sigh, smh]"
3,32003,"Lordy, my daughter just said, “I wonder how th...",,"[applause, oh_snap, omg, scared, shocked, smh]"
4,32004,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...,"[applause, omg, popcorn, scared, shocked, yes]"


In [62]:
df_dev_ori.to_json('./results/dev.json', orient='records', lines=True)

#### Count predicted class 

In [63]:
total_classes_dev = 0
categories_dict_dev = defaultdict(lambda: 0)
for i in range(len(df_dev_ori)):
    for each_category in df_dev_ori['categories'][i]:
        categories_dict_dev[each_category] += 1
        total_classes_dev += 1
sorted(categories_dict_dev.items(), key=lambda d: d[1], reverse=True) 

[('applause', 1864),
 ('yes', 1862),
 ('agree', 1779),
 ('seriously', 1389),
 ('smh', 1269),
 ('facepalm', 1127),
 ('no', 1030),
 ('sigh', 1029),
 ('shocked', 833),
 ('dance', 828),
 ('slow_clap', 799),
 ('idk', 757),
 ('hug', 713),
 ('omg', 701),
 ('awww', 686),
 ('happy_dance', 666),
 ('good_luck', 646),
 ('you_got_this', 572),
 ('hearts', 558),
 ('eye_roll', 474),
 ('shrug', 439),
 ('thank_you', 416),
 ('please', 408),
 ('oops', 365),
 ('scared', 363),
 ('oh_snap', 346),
 ('popcorn', 303),
 ('thumbs_up', 302),
 ('wink', 300),
 ('eww', 260),
 ('win', 214),
 ('high_five', 153),
 ('want', 140),
 ('do_not_want', 138),
 ('ok', 123),
 ('kiss', 68),
 ('yawn', 44),
 ('sorry', 36)]

## Power n weighted sum to blend models, average same model type first

In [11]:
import csv
from os import listdir
from os.path import isfile
from operator import add 

### Evaluate training data

In [131]:
to_predict = []
for i in range(len(df_new)):
    # for roberta format
    text_reply = '<s> ' + df_new['text_a'][i] + ' </s></s> ' + df_new['text_b'][i] + ' <s>'
#     for bert format
#     text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

#### Predict all roberta models (need to change path)

In [130]:
model_roberta_base = MultiLabelClassificationModel('roberta', 'model_results_gpu/formal_roberta_training_preprocessed_#112/')
predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)

I0607 14:08:24.240644 139831797843776 configuration_utils.py:283] loading configuration file model_results_gpu/formal_roberta_training_preprocessed_#112/config.json
I0607 14:08:24.241528 139831797843776 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

In [133]:
with open("./round2_results/roberta_10.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(raw_outputs_roberta_base)

#### Predict all cased and uncased bert models (need to change path)

In [135]:
for model_idx in range(1, 6):
    model_bert_base = MultiLabelClassificationModel('bert', 'model_results_gpu/formal_bert_uncased_LM_training_preprocessed_' + str(model_idx) + '/')
    to_predict = []
    for i in range(len(df_new)):
        # for roberta format
#         text_reply = '<s> ' + df_new['text_a'][i] + ' </s></s> ' + df_new['text_b'][i] + ' <s>'
    #     for bert format
        text_reply = '[CLS] ' + df_new['text_a'][i] + ' [SEP] ' + df_new['text_b'][i] + ' [SEP]'
        to_predict.append(text_reply)
    predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
    with open("./final_results/bert_uncase_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_bert_base)

I0607 14:23:49.151986 139831797843776 configuration_utils.py:283] loading configuration file model_results_gpu/formal_bert_uncased_LM_training_preprocessed_1/config.json
I0607 14:23:49.153757 139831797843776 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "

I0607 14:25:48.131691 139831797843776 configuration_utils.py:283] loading configuration file model_results_gpu/formal_bert_uncased_LM_training_preprocessed_2/config.json
I0607 14:25:48.132866 139831797843776 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "

I0607 14:27:47.512206 139831797843776 configuration_utils.py:283] loading configuration file model_results_gpu/formal_bert_uncased_LM_training_preprocessed_3/config.json
I0607 14:27:47.513008 139831797843776 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "

I0607 14:29:46.789490 139831797843776 configuration_utils.py:283] loading configuration file model_results_gpu/formal_bert_uncased_LM_training_preprocessed_4/config.json
I0607 14:29:46.790417 139831797843776 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "

I0607 14:31:46.289960 139831797843776 configuration_utils.py:283] loading configuration file model_results_gpu/formal_bert_uncased_LM_training_preprocessed_5/config.json
I0607 14:31:46.291116 139831797843776 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "

In [204]:
files = listdir('./final_results/')
roberta_prob = np.zeros([len(df_new), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_new), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_new), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./final_results/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_case_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncase') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

### Sum up all probabilities and average

In [205]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

### Power 3.5 of weighted sum

In [206]:
P = 3.5
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    # Check correctness
    correct = 0
    for correct_class in df['categories'][eid]:
        if correct_class in predict_class:
            correct += 1
    total_predict.append(predict_class)
    avg_map += (correct / len(df['categories'][eid]))
print("avg map = {} ({} / {})".format(avg_map / len(df['categories']), avg_map, len(df['categories'])))

avg map = 0.715995306038411 (22911.84979322915 / 32000)


### Predicting dev data

In [41]:
to_predict = []
for i in range(len(df_dev_result)):
    # for roberta format
    text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
#     for bert format
#     text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [42]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round2_blend_model/round2_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./round2_blend_model/round2_results/dev/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0615 10:57:00.572922 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training1/config.json
I0615 10:57:00.574002 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:57:17.109853 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training2/config.json
I0615 10:57:17.110505 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:57:33.760521 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/config.json
I0615 10:57:33.761148 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:57:50.134470 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training4/config.json
I0615 10:57:50.135174 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:58:06.542863 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training5/config.json
I0615 10:58:06.543573 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:58:23.177581 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training6/config.json
I0615 10:58:23.178209 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:58:39.549983 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training7/config.json
I0615 10:58:39.550599 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:58:55.991571 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training8/config.json
I0615 10:58:55.992190 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:59:12.433299 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training9/config.json
I0615 10:59:12.434004 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 10:59:29.021896 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training10/config.json
I0615 10:59:29.022513 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

In [43]:
to_predict = []
for i in range(len(df_dev_result)):
    # for roberta format
#     text_reply = '<s> ' + df_dev_result['text_a'][i] + ' </s></s> ' + df_dev_result['text_b'][i] + ' <s>'
#     for bert format
    text_reply = '[CLS] ' + df_dev_result['text_a'][i] + ' [SEP] ' + df_dev_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [44]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./round2_blend_model/round2_results/dev/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0615 11:00:31.592876 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training1/config.json
I0615 11:00:31.593585 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:00:47.438678 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training2/config.json
I0615 11:00:47.439318 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:01:03.401802 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training3/config.json
I0615 11:01:03.402525 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:01:19.253464 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/config.json
I0615 11:01:19.254083 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:01:35.097242 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training5/config.json
I0615 11:01:35.097864 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:01:51.124197 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training1/config.json
I0615 11:01:51.124887 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:02:07.021936 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training2/config.json
I0615 11:02:07.022541 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:02:22.912891 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training3/config.json
I0615 11:02:22.913564 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:02:38.809378 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training4/config.json
I0615 11:02:38.810037 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:02:54.901561 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/config.json
I0615 11:02:54.902267 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

In [112]:
files = listdir('./round2_blend_model/round2_results/dev/')
roberta_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_test_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./round2_blend_model/round2_results/dev/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_case_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncase') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [113]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [114]:
P = 1.8
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [115]:
df_dev_ori['categories'] = total_predict
df_dev_ori.head()

,idx,text,reply,categories
0,32000,"Drop your cash app, use hashtag #BailoutHumansNow",$tyratomaro #BailoutHumans,"[applause, dance, good_luck, happy_dance, plea..."
1,32001,After interviewing with a few incredible peopl...,CONGRATS!!!!!,"[agree, applause, dance, happy_dance, slow_cla..."
2,32002,I know GTC festival not happening next month b...,,"[agree, facepalm, no, sigh, smh, yes]"
3,32003,"Lordy, my daughter just said, “I wonder how th...",,"[facepalm, omg, seriously, shocked, sigh, smh]"
4,32004,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...,"[agree, applause, no, popcorn, sigh, yes]"


In [116]:
df_dev_ori.to_json('./round2_results/dev.json', orient='records', lines=True)

### Predicting test data

In [50]:
to_predict = []
for i in range(len(df_test_result)):
    # for roberta format
    text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
#     for bert format
#     text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [51]:
for model_idx in range(1, 11):
    model_roberta_base = MultiLabelClassificationModel('roberta', 'round2_blend_model/round2_MLR/outputs_roberta_LM_training' + str(model_idx) + '/')
    predictions_roberta_base, raw_outputs_roberta_base = model_roberta_base.predict(to_predict)
    with open("./round2_blend_model/round2_results/test/roberta_" + str(model_idx) + ".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(raw_outputs_roberta_base)

I0615 11:07:37.106949 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training1/config.json
I0615 11:07:37.107669 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:07:53.573905 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training2/config.json
I0615 11:07:53.574656 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:08:09.962673 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/config.json
I0615 11:08:09.963291 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:08:12.493335 140378694625088 tokenization_utils.py:929] Model name 'round2_blend_model/round2_MLR/outputs_roberta_LM_training3/' not found in model shortcut name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). Assuming 'round2_blend_model/round2_MLR/outputs_roberta_LM_training3/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0615 11:08:12.493974 140378694625088 tokenization_utils.py:958] Didn't find file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/added_tokens.json. We won't load it.
I0615 11:08:12.494538 140378694625088 tokenization_utils.py:1013] loading file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/vocab.json
I0615 11:08:12.494816 140378694625088 tokenization_utils.py:1013] loading file round2_blend_model/round2_MLR/outputs_roberta_LM_training3/merges.txt
I0615 11:08:12.495084 140378694625088 tokenization_utils.py:1013

I0615 11:08:26.358447 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training4/config.json
I0615 11:08:26.359086 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

...

I0615 11:08:59.294439 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training6/config.json
I0615 11:08:59.295136 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:09:15.665878 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training7/config.json
I0615 11:09:15.666563 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:09:32.071654 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training8/config.json
I0615 11:09:32.072266 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:09:48.618046 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training9/config.json
I0615 11:09:48.618675 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23

I0615 11:10:05.035332 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_roberta_LM_training10/config.json
I0615 11:10:05.035945 140378694625088 configuration_utils.py:321] Model config RobertaConfig {
  "architectures": [
    "RobertaForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "2

In [52]:
to_predict = []
for i in range(len(df_test_result)):
    # for roberta format
#     text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
#     for bert format
    text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [53]:
for bert_type in ['uncased', 'cased']:
    for model_idx in range(1, 6):
        model_bert_base = MultiLabelClassificationModel('bert', 'round2_blend_model/round2_MLR/outputs_bert_' + bert_type + '_LM_training' + str(model_idx) + '/')
        predictions_bert_base, raw_outputs_bert_base = model_bert_base.predict(to_predict)
        with open("./round2_blend_model/round2_results/test/bert_" + bert_type + "_" + str(model_idx) + ".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(raw_outputs_bert_base)

I0615 11:14:29.435397 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training1/config.json
I0615 11:14:29.436059 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:14:45.214638 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training2/config.json
I0615 11:14:45.215296 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:15:01.142184 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training3/config.json
I0615 11:15:01.142876 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:15:16.941130 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/config.json
I0615 11:15:16.941755 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "2

I0615 11:15:18.688717 140378694625088 tokenization_utils.py:929] Model name 'round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming 'round2_blend_model/round2_MLR/outputs_bert_uncased_LM_training4/' is a path, a model identifier, or url to a directory containing tokenizer files.
I0615 11:15:18.689321 140378694625088 tokenization_utils.py:958] Didn't find file round2_blend_model/round2_MLR/outputs_be

...

I0615 11:16:04.541762 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training2/config.json
I0615 11:16:04.542450 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:16:20.400134 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training3/config.json
I0615 11:16:20.400911 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

I0615 11:16:36.364712 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training4/config.json


I0615 11:16:36.365415 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    "30": "LABEL_30",
    "31": "LABEL_31",
    "32": "LABEL_32",

I0615 11:16:52.200727 140378694625088 configuration_utils.py:283] loading configuration file round2_blend_model/round2_MLR/outputs_bert_cased_LM_training5/config.json
I0615 11:16:52.201379 140378694625088 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMultiLabelSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25"

In [54]:
files = listdir('./round2_blend_model/round2_results/test/')
roberta_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_uncase_prob = np.zeros([len(df_test_result), len(categories_mapping)])
bert_case_prob = np.zeros([len(df_test_result), len(categories_mapping)])
for f in files:
    if f.find('.') != 0:
        model_prob = pd.read_csv('./round2_blend_model/round2_results/test/' + f, header=None)
        probability_list = model_prob.values.tolist()
        if f.find('roberta') >= 0:
            roberta_prob = np.add(roberta_prob, probability_list)
        if f.find('_case_') >= 0:
            bert_case_prob = np.add(bert_case_prob, probability_list)
        if f.find('uncase') >= 0:
            bert_uncase_prob = np.add(bert_uncase_prob, probability_list)

In [55]:
roberta_count = 10
bert_count = 5
roberta_prob /= roberta_count
bert_case_prob /= bert_count
bert_uncase_prob /= bert_count

In [56]:
P = 1.8
total_predict = []
for eid, row_roberta_base in enumerate(roberta_prob):
    row = (row_roberta_base ** P) * 3.0 + \
            (bert_case_prob[eid] ** P) * 0.8 + (bert_uncase_prob[eid] ** P) * 0.8 
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
            
    total_predict.append(predict_class)

In [57]:
df_test_ori['categories'] = total_predict
df_test_ori.head()

,idx,text,reply,categories
0,36000,@Youngdeji_ I think if uzi and carti dropping ...,,"[agree, dance, happy_dance, no, please, yes]"
1,36001,For the third year in a row we’re discussing t...,,"[eye_roll, facepalm, no, seriously, sigh, smh]"
2,36002,dababy album sounds like it was made for nigga...,That's why you bought it.,"[agree, idk, seriously, shrug, smh, yes]"
3,36003,Majority of Indians don’t watch any sport othe...,@ZairaWasimmm got a great story because of the...,"[agree, applause, awww, shocked, slow_clap, th..."
4,36004,everybody is just now listening to @madisonbee...,,"[agree, facepalm, idk, seriously, sigh, smh]"


In [60]:
df_test_ori.to_json('./round2_results/eval.json', orient='records', lines=True)

## Predict testing data

In [30]:
to_predict = []
for i in range(len(df_test_result)):
    # for roberta format
    text_reply = '<s> ' + df_test_result['text_a'][i] + ' </s></s> ' + df_test_result['text_b'][i] + ' <s>'
#     for bert format
#     text_reply = '[CLS] ' + df_test_result['text_a'][i] + ' [SEP] ' + df_test_result['text_b'][i] + ' [SEP]'
    to_predict.append(text_reply)

In [31]:
predictions, raw_outputs = model.predict(to_predict)

I0606 08:03:32.224428 140151191160640 classification_model.py:801]  Converting to features started. Cache is not used.


In [32]:
total_predict = []
for eid, row in enumerate(raw_outputs):
    predict_class = []
    correct = 0
    sort_index = sorted(range(len(row)), key=lambda k: row[k], reverse=True)
    for key, value in categories_mapping.items():
        if value in sort_index[0:6]:
            predict_class.append(key)
    total_predict.append(predict_class)

In [33]:
df_test_ori['categories'] = total_predict

In [34]:
df_test_ori.head()

,idx,text,reply,categories
0,36000,@Youngdeji_ I think if uzi and carti dropping ...,,"[agree, awww, dance, happy_dance, wink, yes]"
1,36001,For the third year in a row we’re discussing t...,,"[facepalm, no, omg, seriously, sigh, smh]"
2,36002,dababy album sounds like it was made for nigga...,That's why you bought it.,"[facepalm, idk, seriously, shrug, sigh, smh]"
3,36003,Majority of Indians don’t watch any sport othe...,@ZairaWasimmm got a great story because of the...,"[agree, applause, awww, sigh, slow_clap, thank..."
4,36004,everybody is just now listening to @madisonbee...,,"[facepalm, omg, seriously, shocked, sigh, smh]"


In [35]:
df_test_ori.to_json('./results/eval.json', orient='records', lines=True)

In [117]:
torch.cuda.empty_cache()